# 9 Head poses Model Training

Neural Network model for a quick Head Pose estimation, using just facial points as input. The model was made for real-time control of a Pan-Tilt camera using face movements as the control signal. 

### Load dependencies, set random seed for reproducibility

In [1]:
import numpy as np
import pandas as pd
np.random.seed(14)

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

from keras.layers import Dropout 
from keras.layers.normalization import BatchNormalization 
from keras import regularizers 

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### Read and preprocess the dataset

In [3]:
dfCenter = pd.read_csv('datasets/face_center.csv')
dfCenter2 = pd.read_csv('datasets/face_center2.csv')
dfCenter3 = pd.read_csv('datasets/face_center3.csv')
dfCenter = dfCenter.append(dfCenter2, ignore_index = True).append(dfCenter3, ignore_index = True)

dfRight = pd.read_csv('datasets/face_right.csv')
dfRight2 = pd.read_csv('datasets/face_right2.csv')
dfRight3 = pd.read_csv('datasets/face_right3.csv')
dfRight4 = pd.read_csv('datasets/face_right4.csv')
dfRight = dfRight.append(dfRight2, ignore_index = True).append(dfRight3, ignore_index = True).append(dfRight4, ignore_index = True)

dfLeft = pd.read_csv('datasets/face_left.csv')
dfLeft2 = pd.read_csv('datasets/face_left2.csv')
dfLeft3 = pd.read_csv('datasets/face_left3.csv')
dfLeft4 = pd.read_csv('datasets/face_left4.csv')
dfLeft5 = pd.read_csv('datasets/face_left5.csv')
dfLeft = dfLeft.append(dfLeft2, ignore_index = True).append(dfLeft3, ignore_index = True).append(dfLeft4, ignore_index = True).append(dfLeft5, ignore_index = True)

dfUp = pd.read_csv('datasets/face_up.csv')
dfDown = pd.read_csv('datasets/face_down.csv')
dfUpRight = pd.read_csv('datasets/face_up-right.csv')
dfUpLeft = pd.read_csv('datasets/face_up-left.csv')
dfDownRight = pd.read_csv('datasets/face_down-right.csv')

dfDownLeft = pd.read_csv('datasets/face_down-left.csv')
dfDownLeft2 = pd.read_csv('datasets/face_down-left2.csv')
dfDownLeft3 = pd.read_csv('datasets/face_down-left3.csv')
dfDownLeft = dfDownLeft.append(dfDownLeft2, ignore_index = True).append(dfDownLeft3, ignore_index = True)

columns = list(dfCenter)

dfCenter['RESULT'] = 0
dfRight['RESULT'] = 1
dfLeft['RESULT'] = 2
dfUp['RESULT'] = 3
dfDown['RESULT'] = 4
dfUpRight['RESULT'] = 5
dfUpLeft['RESULT'] = 6
dfDownRight['RESULT'] = 7
dfDownLeft['RESULT'] = 8


df = dfCenter.append(dfRight, ignore_index=True).append(dfLeft, ignore_index = True)
df = df.append(dfUp, ignore_index = True).append(dfDown, ignore_index = True)
df = df.append(dfUpRight, ignore_index = True).append(dfUpLeft, ignore_index = True)
df = df.append(dfDownRight, ignore_index = True).append(dfDownLeft, ignore_index = True)


print(df.shape)

(7314, 61)


In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,RESULT
0,0.946330,0.840339,0.756160,0.720585,0.707754,0.716760,0.797062,0.852249,0.892855,0.873837,...,0.227869,0.254465,0.322255,0.432351,0.492460,0.499847,0.494317,0.475419,0.454395,0
1,0.856706,0.758474,0.642529,0.584934,0.602656,0.673690,0.769424,0.845610,0.860791,0.847841,...,0.259727,0.233466,0.293848,0.394922,0.456411,0.488397,0.513183,0.507366,0.488397,0
2,0.878858,0.760248,0.657293,0.598558,0.605994,0.665656,0.771598,0.842121,0.852072,0.811534,...,0.261712,0.229537,0.283921,0.379951,0.441522,0.459073,0.464209,0.486379,0.472645,0
3,0.896369,0.769889,0.679754,0.600755,0.615911,0.681689,0.771598,0.842121,0.852072,0.811534,...,0.261712,0.230681,0.283921,0.379951,0.441522,0.482027,0.486921,0.501315,0.492836,0
4,0.889466,0.769424,0.670478,0.582623,0.592269,0.657063,0.769424,0.823298,0.837595,0.824935,...,0.259727,0.233466,0.293848,0.394922,0.451059,0.465194,0.490052,0.485071,0.488397,0


### Define input data and desired output, and then split training and test observations

In [5]:
X = df[columns]
y = df['RESULT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=14)

X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')

n_classes = 9
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

### Define Neural Network architecture

In [6]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(60,)))
model.add(BatchNormalization())


model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(9, activation='softmax'))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               7808      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

### Configure the model

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00125), metrics=['accuracy'])

### Train the model

In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 5119 samples, validate on 2195 samples
Epoch 1/100
5119/5119 [==============================] - 1s 161us/step - loss: 0.1224 - acc: 0.9576 - val_loss: 0.1069 - val_acc: 0.9658
Epoch 2/100
5119/5119 [==============================] - 1s 155us/step - loss: 0.1199 - acc: 0.9578 - val_loss: 0.1176 - val_acc: 0.9567
Epoch 3/100
5119/5119 [==============================] - 1s 153us/step - loss: 0.1132 - acc: 0.9598 - val_loss: 0.0994 - val_acc: 0.9626
Epoch 4/100
5119/5119 [==============================] - 1s 168us/step - loss: 0.1174 - acc: 0.9603 - val_loss: 0.1314 - val_acc: 0.9581
Epoch 5/100
5119/5119 [==============================] - 1s 172us/step - loss: 0.1163 - acc: 0.9582 - val_loss: 0.0872 - val_acc: 0.9686
Epoch 6/100
5119/5119 [==============================] - 1s 162us/step - loss: 0.1240 - acc: 0.9562 - val_loss: 0.0945 - val_acc: 0.9622
Epoch 7/100
5119/5119 [==============================] - 1s 170us/step - loss: 0.1222 - acc: 0.9557 - val_loss: 0.0779 - val_acc: 

### Saving the model

In [14]:
model.save('model/model_9_positions.h5')